In [2]:
import os
import csv
import pandas as pd
import numpy as np

## Analysis on HMDA file only

- todo - this is only the mini because that's all I have on my comp
- todo - write up findings after this is run on all... For a table... what's the takeaway we (Matt) should look at 


In [19]:
hmda = pd.read_csv('../input_data/state_AZ-CA.csv')

# Our tables will contract BMO and BOW to all other banks
def lei_map(lei):
    if lei == 'QOT5WN9RBKQTFRVKEV31':
        return 'Bank of West'
    elif lei == '3Y4U8VZURTYWI1W2K376':
        return 'BMO Harris'
    elif lei == np.nan:
        return np.nan
    else: 
        return 'All Other Banks'

hmda['which_bank'] = hmda['lei'].map(lei_map)

hmda['approved'] = (hmda['action_taken'] == 2) | (hmda['action_taken'] == 1) 

hmda['application'] = (hmda['action_taken'] < 4 ) 

loan_type_mapping = {
    1: 'Conventional',
    2: 'FHA-insured',
    3: 'VA-guaranteed',
    4: 'FSA/RHS-guaranteed',
}

hmda['loan_type'] = hmda['loan_type'].map(lambda x: loan_type_mapping.get(x, x))

loan_purpose_mapping = {
    1: 'Home purchase',
    2: 'Home improvement',
    31: 'Refinancing',
    32: 'Cash-out refinancing',
    4: 'Other Purpose',
    5: 'Not applicable',
}

hmda['loan_purpose'] = hmda['loan_purpose'].map(lambda x: loan_purpose_mapping.get(x, x))


C:\Users\twsca\AppData\Local\Temp\ipykernel_5408\2282250758.py:1: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,43,44,46,77,78,79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  hmda = pd.read_csv('../input_data/state_AZ-CA.csv')


## Denial rates by product and purpose (aggregate)

#Application do we want to count the pre approvals as an application? 
    # 7 reapproval request denied - application or denial? 
    # 8 preapproval approved but not accepted 
    # 1 originated, means that the loan was accepted and paid out? 
    # 4 does application withdrawn by applicant count as a completed application, that could affect the denial rate, or should that not be considered?

In [8]:
# the denial rate = denials/apps
# so restrict to applications

apps = hmda.query('action_taken < 4').copy()

apps['denial'] = apps['action_taken'] == 3

pivot = apps.pivot_table(index=['loan_type', 'loan_purpose'], columns = ['which_bank'], values='denial', aggfunc='mean')

pivot['diff_BMO'] = pivot['BMO Harris'] - pivot['All Other Banks']
pivot['diff_BOW'] = pivot['Bank of West'] - pivot['All Other Banks']
pivot.style.format('{:,.2f}')

## Avg requested loan size by product and purpose for all applications (aggregate)

In [17]:
hmda.pivot_table(index=['loan_type', 'loan_purpose'], 
                 columns = ['which_bank'], 
                 values='loan_amount', 
                 aggfunc='mean').style.format('{:,.0f}')

## Avg loan size by product and purpose for approved loans (aggregate)

In [18]:
hmda.query('approved == 1').pivot_table(index=['loan_type', 'loan_purpose'], 
                 columns = ['which_bank'], 
                 values='loan_amount', 
                 aggfunc='mean').style.format('{:,.0f}')

## Rate spread by product and purpose (aggregate)

## Median income of applicants by product and purpose (aggregate)

In [5]:
hmda.pivot_table(index=['loan_type', 'loan_purpose'], 
                 columns = ['which_bank'], 
                 values='income', 
                 aggfunc='mean').style.format('{:,.2f}')

## Median income of approved loans by product and purpose (aggregate)

In [6]:
hmda.query('approved == 1').pivot_table(index=['loan_type', 'loan_purpose'], 
                 columns = ['which_bank'], 
                 values='income', 
                 aggfunc='median').style.format('{:,.2f}')

## Brainstorming: Tables and figures we could make

- State-level versions of the above to see if there is a different set of facts in either...
- Revisit the project descript doc and the whiteboard picture...
- todo... what else?

- looking at MSA vs census tract, we noticed that census tracts will be more similar in population characteristics, but would it be helpfl or useful to use grouping by MSA to see loan info in metropolitan areas?
- would looking at denial rates by occupancy type within census tracts be an interesting statistic? It would be able to see if within individual census tracts the type of occupancy the loan os for and if the denial rates are being artificially increased or decreased by loans of a specific occupancy type
- in teh project description it says that we should determine whether any competitors have special lending programs in order to target underserved customer, is this something to research through google/competitors websites, or is there a specific way programs like this are discovered?

In [20]:
pivot1 = apps.pivot_table(index=['loan_type', 'loan_purpose'], columns = ['which_bank'], values='denial', aggfunc='mean')
pivot2 = hmda.query('approved == 1').pivot_table(index=['loan_type', 'loan_purpose'], columns = ['which_bank'], values='loan_amount', aggfunc='mean')
pivot3 = hmda.pivot_table(index=['loan_type', 'loan_purpose'], columns = ['which_bank'], values='income', aggfunc='mean')
pivot4 = hmda.query('approved == 1').pivot_table(index=['loan_type', 'loan_purpose'], columns = ['which_bank'], values='income', aggfunc='median')

pivot = pivot1.merge(pivot2, left_index=True, right_index=True, how='outer')
pivot = pivot.merge(pivot3, left_index=True, right_index=True, how='outer')
pivot = pivot.merge(pivot4, left_index=True, right_index=True, how='outer')


pivot.style.format('{:,.2f}')


C:\Users\twsca\AppData\Local\Temp\ipykernel_5408\1720357269.py:8: FutureWarning: Passing 'suffixes' which cause duplicate columns {'All Other Banks_x', 'BMO Harris_x', 'Bank of West_x'} in the result is deprecated and will raise a MergeError in a future version.
  pivot = pivot.merge(pivot4, left_index=True, right_index=True, how='outer')


pivot.rename(columns={'one': 'All Other Banks Denial Rate', 'two': 'BMO Harris Denial Rate', 'three': 'Bank of West Denial Rate', 'four': 'All Other Banks Avg Loan Size for All Applicants'
